In [7]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
# school_data
# student_data

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [8]:
studentCount = school_data_complete_df["student_name"].count()
schoolsList = school_data_complete_df["school_name"].unique()
schoolCount = int(len(schoolsList))
readAvg = school_data_complete_df["reading_score"].mean()
mathAvg = school_data_complete_df["math_score"].mean()
mathPassing = school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70]
readPassing = school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 70]
overallPassing = school_data_complete_df.loc[(school_data_complete_df["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]


totalBudget = school_data["budget"].sum()
mathPercent = len(mathPassing)/studentCount
readPercent = len(readPassing)/studentCount
overallPercent = len(overallPassing)/studentCount
# mathPercent
# readPercent
# overallPercent

districtSummary_df = pd.DataFrame({"Total Schools": [schoolCount],
                                   "Total Students":[studentCount],
                                   "Total Budget": totalBudget,
                                   "Average Math Score": [mathAvg], 
                                   "Average Reading Score": [readAvg], 
                                   "% Passing Math": mathPercent,
                                   "% Passing Reading":readPercent, 
                                   "% Overall Passing":overallPercent})
districtSummary_df





,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,0.749809,0.858055,0.651723


In [9]:
reduced_school_data = school_data[["school_name","type","size","budget"]]
schoolGroup_df = reduced_school_data.set_index("school_name").sort_values("school_name",ascending = True)
studentBudget = schoolGroup_df["budget"]/schoolGroup_df["size"]
schoolGroup_df["Per Student Budget"] = studentBudget

schoolGroup_df

,type,size,budget,Per Student Budget
school_name,,,,
Bailey High School,District,4976,3124928,628.0
Cabrera High School,Charter,1858,1081356,582.0
Figueroa High School,District,2949,1884411,639.0
Ford High School,District,2739,1763916,644.0
Griffin High School,Charter,1468,917500,625.0
Hernandez High School,District,4635,3022020,652.0
Holden High School,Charter,427,248087,581.0
Huang High School,District,2917,1910635,655.0
Johnson High School,District,4761,3094650,650.0


In [12]:
#Average columns for math and reading scores
reduced_student_data = student_data[["school_name", "math_score", "reading_score"]]
scoreGroup_df = reduced_student_data.groupby("school_name").mean()
# scoreGroup_df

#Percentages columns for math, reading and overall
##Find passing students
mathSchoolPassing = reduced_student_data.loc[reduced_student_data["math_score"] >= 70]
readSchoolPassing = reduced_student_data.loc[reduced_student_data["reading_score"] >= 70]
overallSchoolPassing = reduced_student_data.loc[(reduced_student_data["reading_score"] >= 70) & (school_data_complete_df["math_score"] >= 70)]

## Calculating Passing DFs 
mathCounts = mathSchoolPassing.groupby("school_name").count()
mathSchoolPercent = mathCounts["math_score"]/schoolGroup_df["size"]

readingCounts = readSchoolPassing.groupby("school_name").count()
readSchoolPercent = readingCounts["reading_score"]/schoolGroup_df["size"]

allCounts = overallSchoolPassing.groupby("school_name").count()
overallSchoolPercent = allCounts["reading_score"]/schoolGroup_df["size"]

passPercentages_df = pd.DataFrame({"% Passing Math": mathSchoolPercent,
                                   "% Passing Reading":readSchoolPercent,
                                   "% Overall Passing": overallSchoolPercent,
                                  })
passPercentages_df

partialSummary_df = pd.merge(schoolGroup_df,scoreGroup_df, on="school_name")
partialSummary_df

schoolSummary_df = pd.merge(partialSummary_df,passPercentages_df, on="school_name")
schoolSummary_df

renamed_schoolSummary_df = schoolSummary_df.rename(columns={"type":"School Type", 
                                                            "size":"Total Students",
                                                            "budget":"Total School Budget",
                                                            "math_score":"Average Math Score",
                                                            "reading_score":"Average Reading Score"
                                                           })
# test = renamed_schoolSummary_df.dropna(how='any')
renamed_schoolSummary_df
# test

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,0.666801,0.819333,0.546423
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,0.683096,0.792990,0.542899
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,0.925059,0.962529,0.892272
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392


In [14]:
top_renamed_schoolSummary_df = renamed_schoolSummary_df.sort_values("% Overall Passing", ascending=False)
top_renamed_schoolSummary_df.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,0.941335,0.970398,0.913348
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,0.932722,0.973089,0.909480
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,0.933924,0.971390,0.905995
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,0.938677,0.965396,0.905826
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,0.945946,0.959459,0.905405


In [15]:
top_renamed_schoolSummary_df = renamed_schoolSummary_df.sort_values("% Overall Passing", ascending=False)
top_renamed_schoolSummary_df.tail(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,0.660576,0.812224,0.535392
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,0.667530,0.808630,0.535275
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,0.656839,0.813164,0.535139
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,0.659885,0.807392,0.532045
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,0.663666,0.802201,0.529882


In [64]:
student_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [95]:
readNine = {}
readTen = {}
readEleven = {}
readTwelve = {}

schools = []
mathScore = []
readScore = []


for i in range(20) :
    if student_data["grade"][i] == "9th":
        student = student_data["student_name"][i]
        schools.append(student_data["school_name"][i])
        readScore.append(student_data["reading_score"][i])
        mathScore.append(student_data["math_score"][i])
        readNine = {"School Name": school,
                    "Reading Score": readScore,
                    "Math Score": mathScore
                   } 
#         print(f'{readNine["School Name"]} {readNine["Reading Score"]} {readNine["Math Score"]}')
print(readNine["School Name"])
print(readNine["Reading Score"])
print(readNine["Math Score"])
test_df = pd.DataFrame(readNine)
test_df

Huang High School
[66, 97, 94, 64, 71, 88, 74, 64, 89]
[79, 84, 94, 79, 79, 55, 75, 69, 80]


,School Name,Reading Score,Math Score
0,Huang High School,66,79
1,Huang High School,97,84
2,Huang High School,94,94
3,Huang High School,64,79
4,Huang High School,71,79
5,Huang High School,88,55
6,Huang High School,74,75
7,Huang High School,64,69
8,Huang High School,89,80


In [66]:
mathScores_df = scoreGroup_df.copy()
mathScores_df

,math_score,reading_score
school_name,,
Bailey High School,77.048432,81.033963
Cabrera High School,83.061895,83.975780
Figueroa High School,76.711767,81.158020
Ford High School,77.102592,80.746258
Griffin High School,83.351499,83.816757
Hernandez High School,77.289752,80.934412
Holden High School,83.803279,83.814988
Huang High School,76.629414,81.182722
Johnson High School,77.072464,80.966394
